In [7]:
from vllm import LLM
llm = LLM(model="deepseek-ai/DeepSeek-V2-Lite", enforce_eager=True)  # 先和你 engine_args 保持一致
out = llm.generate({"prompt": "Hello"}, sampling_params={"max_tokens": 16})
print(out[0].outputs[0].text)

INFO 11-07 02:43:57 [__init__.py:216] Automatically detected platform cuda.


ImportError: /home/lkx/data/qwen_moe/.venv/lib/python3.13/site-packages/vllm/_C.abi3.so: undefined symbol: _ZNK3c106SymInt6sym_neERKS0_

In [6]:
from optimum_benchmark import Benchmark, BenchmarkConfig, InferenceConfig, VLLMConfig, LlamaCppConfig, PyTorchConfig, TorchrunConfig, ProcessConfig

from optimum_benchmark.logging_utils import setup_logging

model_id = "deepseek-ai/DeepSeek-V2-Lite"

backend = None

vllm_config = VLLMConfig(
    model=model_id,
    device="cuda",
    device_ids="0",
    version="0.11.0",
    no_weights=False,
    serving_mode="online",
    engine_args={"enforce_eager": True},
)

llama_cpp_config = LlamaCppConfig(
    model=model_id,
    device="cpu",
    no_weights=False,
    task="text-generation",
    filename="./deepseek_V2_Lite.gguf"
)

torch_config = PyTorchConfig(
    model=model_id, device="cuda", device_ids="0", no_weights=False, torch_dtype="bfloat16")

backend_name = 'vllm'
if backend_name == 'vllm':
    backend = vllm_config
elif backend_name == 'llama_cpp':
    backend = llama_cpp_config
elif backend_name == 'pytorch':
    backend = torch_config


setup_logging(level="INFO")
launcher_config = ProcessConfig(
    device_isolation=True, device_isolation_action="kill")
scenario_config = InferenceConfig(
    latency=True, memory=True, energy=True, warmup_runs=2,
    input_shapes={
        "batch_size": 1,
        "sequence_length": 64
    },
    generate_kwargs={
        "max_new_tokens": 64,
        "min_new_tokens": 64
    })
benchmark_config = BenchmarkConfig(
    name="pytorch_qwen",
    scenario=scenario_config,
    launcher=launcher_config,
    backend=backend,
)
benchmark_report = Benchmark.launch(benchmark_config)
benchmark = Benchmark(config=benchmark_config, report=benchmark_report)
benchmark.save_json(f"{backend_name}_{model_id}_benchmark.json")

[2025-11-07 02:43:13,684][backend][INFO] - CUDA_VISIBLE_DEVICES was set to 0.
[2025-11-07 02:43:15,090][backend][INFO] - CUDA_VISIBLE_DEVICES was set to 0.
[2025-11-07 02:43:15,155][process][INFO] - Allocated process launcher
[2025-11-07 02:43:17,991][process][INFO] - 	+ Isolating device(s) [0] for process [2699401] and its children
[2025-11-07 02:43:17,992][process][INFO] - 	+ Executing action [kill] in case of violation
INFO 11-07 02:43:18 [__init__.py:216] Automatically detected platform cuda.
[ISOLATED-PROCESS][2025-11-07 02:43:18,084][hydra.utils][ERROR] - Error getting class at optimum_benchmark.backends.vllm.backend.VLLMBackend: Error loading 'optimum_benchmark.backends.vllm.backend.VLLMBackend':
ImportError('/home/lkx/data/qwen_moe/.venv/lib/python3.13/site-packages/vllm/_C.abi3.so: undefined symbol: _ZNK3c106SymInt6sym_neERKS0_')
[ISOLATED-PROCESS][2025-11-07 02:43:18,085][process][ERROR] - 	+ Sending traceback string to main process
[2025-11-07 02:43:18,087][process][INFO] - 

ChildProcessError: Traceback (most recent call last):
  File "/home/lkx/data/qwen_moe/.venv/lib/python3.13/site-packages/hydra/_internal/utils.py", line 644, in _locate
    obj = getattr(obj, part)
AttributeError: module 'optimum_benchmark.backends.vllm' has no attribute 'backend'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lkx/data/qwen_moe/.venv/lib/python3.13/site-packages/hydra/_internal/utils.py", line 650, in _locate
    obj = import_module(mod)
  File "/home/lkx/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/importlib/__init__.py", line 88, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load